In [ ]:
#default_exp basics

In [ ]:
#export
from fastcore.imports import *

In [ ]:
from fastcore.test import *
from nbdev.showdoc import *
from fastcore.nb_imports import *

# Basic functionality

> Basic functionality used in the fastai library

## Basics

In [ ]:
# export
defaults = SimpleNamespace()

In [ ]:
# export
def ifnone(a, b):
    "`b` if `a` is None else `a`"
    return b if a is None else a

Since `b if a is None else a` is such a common pattern, we wrap it in a function. However, be careful, because python will evaluate *both* `a` and `b` when calling `ifnone` (which it doesn't do if using the `if` version directly).

In [ ]:
test_eq(ifnone(None,1), 1)
test_eq(ifnone(2   ,1), 2)

In [ ]:
#export
def maybe_attr(o, attr):
    "`getattr(o,attr,o)`"
    return getattr(o,attr,o)

Return the attribute `attr` for object `o`.  If the attribute doesn't exist, then return the object `o` instead. 

In [ ]:
class myobj: myattr='foo'

test_eq(maybe_attr(myobj, 'myattr'), 'foo')
test_eq(maybe_attr(myobj, 'another_attr'), myobj)

In [ ]:
#export
def basic_repr(flds=None):
    if isinstance(flds, str): flds = re.split(', *', flds)
    flds = list(flds or [])
    def _f(self):
        sig = ', '.join(f'{o}={maybe_attr(getattr(self,o), "__name__")}' for o in flds)
        return f'{self.__class__.__name__}({sig})'
    return _f

Lookup a user-supplied list of attributes (`flds`) of an object and generate a string with the name of each attribute and its corresponding value. The format of this string is `key=value`, where `key` is the name of the attribute, and `value` is the value of the attribute.  For each value, attempt to use the `__name__` attribute, otherwise fall back to using the value's `__repr__` when constructing the string.  

In [ ]:
class SomeClass:
    a=1
    b='foo'
    __repr__=basic_repr('a,b')
    __name__='some-class'
    
class AnotherClass:
    c=SomeClass()
    d='bar'
    __repr__=basic_repr(['c', 'd'])
    
sc = SomeClass()    
ac = AnotherClass()

test_eq(repr(sc), 'SomeClass(a=1, b=foo)')
test_eq(repr(ac), 'AnotherClass(c=some-class, d=bar)')

In [ ]:
#export
def is_array(x):
    "`True` if `x` supports `__array__` or `iloc`"
    return hasattr(x,'__array__') or hasattr(x,'iloc')

In [ ]:
is_array(np.array(1)),is_array([1])

(True, False)

In [ ]:
#export
def listify(o):
    "Convert `o` to a `list`"
    if o is None: return []
    if isinstance(o, list): return o
    if isinstance(o, str) or is_array(o): return [o]
    if is_iter(o): return list(o)
    return [o]

Conversion is designed to "do what you mean", e.g:

In [ ]:
[(o,listify(o)) for o in
 ('hi',array(1),1, [1,2], range(3))]

[('hi', ['hi']),
 (array(1), [array(1)]),
 (1, [1]),
 ([1, 2], [1, 2]),
 (range(0, 3), [0, 1, 2])]

Generators are turned into lists too:

In [ ]:
gen = (o for o in range(3))
listify(gen)

[0, 1, 2]

In [ ]:
#export
def true(x):
    "Test whether `x` is truthy; collections with >0 elements are considered `True`"
    try: return bool(len(x))
    except: return bool(x)

In [ ]:
[(o,true(o)) for o in
 (array(0),array(1),array([0]),array([0,1]),1,0,'',None)]

[(array(0), False),
 (array(1), True),
 (array([0]), True),
 (array([0, 1]), True),
 (1, True),
 (0, False),
 ('', False),
 (None, False)]

In [ ]:
#export
class NullType:
    "An object that is `False` and can be called, chained, and indexed"
    def __getattr__(self,*args):return null
    def __call__(self,*args, **kwargs):return null
    def __getitem__(self, *args):return null
    def __bool__(self): return False

null = NullType()

In [ ]:
bool(null.hi().there[3])

False

In [ ]:
#export
def tonull(x):
    "Convert `None` to `null`"
    return null if x is None else x

In [ ]:
bool(tonull(None).hi().there[3])

False

In [ ]:
#export
def get_class(nm, *fld_names, sup=None, doc=None, funcs=None, **flds):
    "Dynamically create a class, optionally inheriting from `sup`, containing `fld_names`"
    attrs = {}
    for f in fld_names: attrs[f] = None
    for f in listify(funcs): attrs[f.__name__] = f
    for k,v in flds.items(): attrs[k] = v
    sup = ifnone(sup, ())
    if not isinstance(sup, tuple): sup=(sup,)

    def _init(self, *args, **kwargs):
        for i,v in enumerate(args): setattr(self, list(attrs.keys())[i], v)
        for k,v in kwargs.items(): setattr(self,k,v)

    all_flds = [*fld_names,*flds.keys()]
    def _eq(self,b):
        return all([getattr(self,k)==getattr(b,k) for k in all_flds])

    if not sup: attrs['__repr__'] = basic_repr(all_flds)
    attrs['__init__'] = _init
    attrs['__eq__'] = _eq
    res = type(nm, sup, attrs)
    if doc is not None: res.__doc__ = doc
    return res

In [ ]:
show_doc(get_class, title_level=4)

<h4 id="get_class" class="doc_header"><code>get_class</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>get_class</code>(**`nm`**, **\*`fld_names`**, **`sup`**=*`None`*, **`doc`**=*`None`*, **`funcs`**=*`None`*, **\*\*`flds`**)

Dynamically create a class, optionally inheriting from `sup`, containing `fld_names`

In [ ]:
_t = get_class('_t', 'a', b=2)
t = _t()
test_eq(t.a, None)
test_eq(t.b, 2)
t = _t(1, b=3)
test_eq(t.a, 1)
test_eq(t.b, 3)
t = _t(1, 3)
test_eq(t.a, 1)
test_eq(t.b, 3)
test_eq(repr(t), '_t(a=1, b=3)')
test_eq(t, pickle.loads(pickle.dumps(t)))

Most often you'll want to call `mk_class`, since it adds the class to your module. See `mk_class` for more details and examples of use (which also apply to `get_class`).

In [ ]:
#export
def mk_class(nm, *fld_names, sup=None, doc=None, funcs=None, mod=None, **flds):
    "Create a class using `get_class` and add to the caller's module"
    if mod is None: mod = sys._getframe(1).f_locals
    res = get_class(nm, *fld_names, sup=sup, doc=doc, funcs=funcs, **flds)
    mod[nm] = res

Any `kwargs` will be added as class attributes, and `sup` is an optional (tuple of) base classes.

In [ ]:
mk_class('_t', a=1, sup=dict)
t = _t()
test_eq(t.a, 1)
assert(isinstance(t,dict))

A `__init__` is provided that sets attrs for any `kwargs`, and for any `args` (matching by position to fields), along with a `__repr__` which prints all attrs. The docstring is set to `doc`. You can pass `funcs` which will be added as attrs with the function names.

In [ ]:
def foo(self): return 1
mk_class('_t', 'a', sup=dict, doc='test doc', funcs=foo)

t = _t(3, b=2)
test_eq(t.a, 3)
test_eq(t.b, 2)
test_eq(t.foo(), 1)
test_eq(t.__doc__, 'test doc')
t

{}

In [ ]:
#export
def wrap_class(nm, *fld_names, sup=None, doc=None, funcs=None, **flds):
    "Decorator: makes function a method of a new class `nm` passing parameters to `mk_class`"
    def _inner(f):
        mk_class(nm, *fld_names, sup=sup, doc=doc, funcs=listify(funcs)+[f], mod=f.__globals__, **flds)
        return f
    return _inner

In [ ]:
@wrap_class('_t', a=2)
def bar(self,x): return x+1

t = _t()
test_eq(t.a, 2)
test_eq(t.bar(3), 4)

In [ ]:
#export
class ignore_exceptions:
    "Context manager to ignore exceptions"
    def __enter__(self): pass
    def __exit__(self, *args): return True

In [ ]:
show_doc(ignore_exceptions, title_level=4)

<h4 id="ignore_exceptions" class="doc_header"><code>class</code> <code>ignore_exceptions</code><a href="" class="source_link" style="float:right">[source]</a></h4>

> <code>ignore_exceptions</code>()

Context manager to ignore exceptions

In [ ]:
with ignore_exceptions(): 
    # Exception will be ignored
    raise Exception

In [ ]:
#export
def exec_local(code, var_name):
    "Call `exec` on `code` and return the var `var_name"
    loc = {}
    exec(code, globals(), loc)
    return loc[var_name]

In [ ]:
test_eq(exec_local("a=1", "a"), 1)

In [ ]:
#export
def risinstance(types, obj=None):
    "Curried `isinstance` but with args reversed"
    if not obj: return partial(risinstance,types)
    return isinstance(obj, types)

In [ ]:
assert risinstance(int, 1)
assert risinstance(int)(1)

## NoOp

These are used when you need a pass-through function.

In [ ]:
show_doc(noop, title_level=4)

<h4 id="noop" class="doc_header"><code>noop</code><a href="https://github.com/fastai/fastcore/tree/master/fastcore/imports.py#L35" class="source_link" style="float:right">[source]</a></h4>

> <code>noop</code>(**`x`**=*`None`*, **\*`args`**, **\*\*`kwargs`**)

Do nothing

In [ ]:
noop()
test_eq(noop(1),1)

In [ ]:
show_doc(noops, title_level=4)

<h4 id="noops" class="doc_header"><code>noops</code><a href="https://github.com/fastai/fastcore/tree/master/fastcore/imports.py#L39" class="source_link" style="float:right">[source]</a></h4>

> <code>noops</code>(**`x`**=*`None`*, **\*`args`**, **\*\*`kwargs`**)

Do nothing (method)

In [ ]:
class _t: foo=noops
test_eq(_t().foo(1),1)

## Infinite Lists

These lists are useful for things like padding an array or adding index column(s) to arrays.

In [ ]:
#export
#hide
class _InfMeta(type):
    @property
    def count(self): return itertools.count()
    @property
    def zeros(self): return itertools.cycle([0])
    @property
    def ones(self):  return itertools.cycle([1])
    @property
    def nones(self): return itertools.cycle([None])

In [ ]:
#export
class Inf(metaclass=_InfMeta):
    "Infinite lists"
    pass

In [ ]:
show_doc(Inf, title_level=4);

<h4 id="Inf" class="doc_header"><code>class</code> <code>Inf</code><a href="" class="source_link" style="float:right">[source]</a></h4>

> <code>Inf</code>()

Infinite lists

`Inf` defines the following properties:
    
- `count: itertools.count()`
- `zeros: itertools.cycle([0])`
- `ones : itertools.cycle([1])`
- `nones: itertools.cycle([None])`

In [ ]:
test_eq([o for i,o in zip(range(5), Inf.count)],
        [0, 1, 2, 3, 4])

test_eq([o for i,o in zip(range(5), Inf.zeros)],
        [0]*5)

test_eq([o for i,o in zip(range(5), Inf.ones)],
        [1]*5)

test_eq([o for i,o in zip(range(5), Inf.nones)],
        [None]*5)

## Operator Functions

In [ ]:
#export
_dumobj = object()
def _oper(op,a,b=_dumobj): return (lambda o:op(o,a)) if b is _dumobj else op(a,b)

def _mk_op(nm, mod):
    "Create an operator using `oper` and add to the caller's module"
    op = getattr(operator,nm)
    def _inner(a, b=_dumobj): return _oper(op, a,b)
    _inner.__name__ = _inner.__qualname__ = nm
    _inner.__doc__ = f'Same as `operator.{nm}`, or returns partial if 1 arg'
    mod[nm] = _inner

In [ ]:
#export
def in_(x, a):
    "`True` if `x in a`"
    return x in a

operator.in_ = in_

In [ ]:
#export
_all_ = ['lt','gt','le','ge','eq','ne','add','sub','mul','truediv','is_','is_not','in_']

In [ ]:
#export
for op in ['lt','gt','le','ge','eq','ne','add','sub','mul','truediv','is_','is_not','in_']: _mk_op(op, globals())

In [ ]:
# test if element is in another
assert in_('c', ('b', 'c', 'a'))
assert in_(4, [2,3,4,5])
assert in_('t', 'fastai')
test_fail(in_('h', 'fastai'))

# use in_ as a partial
assert in_('fastai')('t')
assert in_([2,3,4,5])(4)
test_fail(in_('fastai')('h'))

In addition to `in_`, the following functions are provided matching the behavior of the equivalent versions in `operator`: *lt gt le ge eq ne add sub mul truediv is_ is_not*.

In [ ]:
lt(3,5),gt(3,5),is_(None,None),in_(0,[1,2])

(True, False, True, False)

Similarly to `_in`, they also have additional functionality: if you only pass one param, they return a partial function that passes that param as the second positional parameter.

In [ ]:
lt(5)(3),gt(5)(3),is_(None)(None),in_([1,2])(0)

(True, False, True, False)

In [ ]:
#export
def true(*args, **kwargs):
    "Predicate: always `True`"
    return True

In [ ]:
assert true(1,2,3)
assert true(False)
assert true(None)
assert true([])

In [ ]:
#export
def stop(e=StopIteration):
    "Raises exception `e` (by default `StopException`)"
    raise e

In [ ]:
#export
def gen(func, seq, cond=true):
    "Like `(func(o) for o in seq if cond(func(o)))` but handles `StopIteration`"
    return itertools.takewhile(cond, map(func,seq))

In [ ]:
test_eq(gen(noop, Inf.count, lt(5)),
        range(5))
test_eq(gen(operator.neg, Inf.count, gt(-5)),
        [0,-1,-2,-3,-4])
test_eq(gen(lambda o:o if o<5 else stop(), Inf.count),
        range(5))

In [ ]:
#export
def chunked(it, chunk_sz=None, drop_last=False, n_chunks=None):
    "Return batches from iterator `it` of size `chunk_sz` (or return `n_chunks` total)"
    assert bool(chunk_sz) ^ bool(n_chunks)
    if n_chunks: chunk_sz = math.ceil(len(it)/n_chunks)
    if not isinstance(it, Iterator): it = iter(it)
    while True:
        res = list(itertools.islice(it, chunk_sz))
        if res and (len(res)==chunk_sz or not drop_last): yield res
        if len(res)<chunk_sz: return

Note that you must pass either `chunk_sz`, or `n_chunks`, but not both.

In [ ]:
t = list(range(10))
test_eq(chunked(t,3),      [[0,1,2], [3,4,5], [6,7,8], [9]])
test_eq(chunked(t,3,True), [[0,1,2], [3,4,5], [6,7,8],    ])

t = map(lambda o:stop() if o==6 else o, Inf.count)
test_eq(chunked(t,3), [[0, 1, 2], [3, 4, 5]])
t = map(lambda o:stop() if o==7 else o, Inf.count)
test_eq(chunked(t,3), [[0, 1, 2], [3, 4, 5], [6]])

t = np.arange(10)
test_eq(chunked(t,3),      [[0,1,2], [3,4,5], [6,7,8], [9]])
test_eq(chunked(t,3,True), [[0,1,2], [3,4,5], [6,7,8],    ])

In [ ]:
#export
def otherwise(x, tst, y):
    "`y if tst(x) else x`"
    return y if tst(x) else x

In [ ]:
test_eq(otherwise(2+1, gt(3), 4), 3)
test_eq(otherwise(2+1, gt(2), 4), 4)

## Attribute Helpers

These functions reduce boilerplate when setting or manipulating attributes or properties of objects.

In [ ]:
#export
def custom_dir(c, add:list):
    "Implement custom `__dir__`, adding `add` to `cls`"
    return dir(type(c)) + list(c.__dict__.keys()) + add

`custom_dir` allows you extract the [`__dict__` property of a class](https://stackoverflow.com/questions/19907442/explain-dict-attribute) and appends the list `add` to it.

In [ ]:
class _T: 
    def f(): pass

s = custom_dir(_T, add=['foo', 'bar']) # a list of everything in `__dict__` of `_T` with ['foo', 'bar'] appended.
assert {'foo', 'bar', 'f'}.issubset(s)

In [ ]:
#export
class AttrDict(dict):
    "`dict` subclass that also provides access to keys as attrs"
    def __getattr__(self,k): return self[k] if k in self else stop(AttributeError(k))
    def __setattr__(self, k, v): (self.__setitem__,super().__setattr__)[k[0]=='_'](k,v)
    def __dir__(self): return custom_dir(self, list(self.keys()))

In [ ]:
show_doc(AttrDict, title_level=4)

<h4 id="AttrDict" class="doc_header"><code>class</code> <code>AttrDict</code><a href="" class="source_link" style="float:right">[source]</a></h4>

> <code>AttrDict</code>() :: `dict`

`dict` subclass that also provides access to keys as attrs

In [ ]:
d = AttrDict(a=1,b="two")
test_eq(d.a, 1)
test_eq(d['b'], 'two')
test_eq(d.get('c','nope'), 'nope')
d.b = 2
test_eq(d.b, 2)
test_eq(d['b'], 2)
d['b'] = 3
test_eq(d['b'], 3)
test_eq(d.b, 3)
assert 'a' in dir(d)

In [ ]:
#export
def with_cast(f):
    "Decorator which uses any parameter annotations as preprocessing functions"
    anno = f.__annotations__
    params = f.__code__.co_varnames[:f.__code__.co_argcount]
    defaults = dict(zip(reversed(params), reversed(f.__defaults__))) if f.__defaults__ else {}
    @functools.wraps(f)
    def _inner(*args, **kwargs):
        args = list(args)
        for i,v in enumerate(params):
            if v in anno:
                c = anno[v]
                if v in kwargs: kwargs[v] = c(kwargs[v])
                elif i<len(args): args[i] = c(args[i])
                elif v in defaults: kwargs[v] = c(defaults[v])
        return f(*args, **kwargs)
    return _inner

In [ ]:
@with_cast
def _f(a, b:Path, c:str='', d=0)->bool: return (a,b,c,d)

test_eq(_f(1, '.', 3), (1,Path('.'),'3',0))
test_eq(_f(1, '.'), (1,Path('.'),'',0))

In [ ]:
#export
def _store_attr(self, anno, **attrs):
    for n,v in attrs.items():
        if n in anno: v = anno[n](v)
        setattr(self, n, v)
        self.__stored_args__[n] = v

In [ ]:
#export
def store_attr(names=None, self=None, but=None, cast=False, **attrs):
    "Store params named in comma-separated `names` from calling context into attrs in `self`"
    fr = sys._getframe(1)
    args = fr.f_code.co_varnames[:fr.f_code.co_argcount]
    if self: args = ('self', *args)
    else: self = fr.f_locals[args[0]]
    if not hasattr(self, '__stored_args__'): self.__stored_args__ = {}
    anno = self.__class__.__init__.__annotations__ if cast else {}
    if attrs: return _store_attr(self, anno, **attrs)
    ns = re.split(', *', names) if names else args[1:]
    _store_attr(self, anno, **{n:fr.f_locals[n] for n in ns if n not in listify(but)})

In it's most basic form, you can use `store_attr` to shorten code like this:

In [ ]:
class T:
    def __init__(self, a,b,c): self.a,self.b,self.c = a,b,c

...to this:

In [ ]:
class T:
    def __init__(self, a,b,c): store_attr('a,b,c', self)

This class behaves as if we'd used the first form:

In [ ]:
t = T(1,c=2,b=3)
assert t.a==1 and t.b==3 and t.c==2

In addition, it stores the attrs as a `dict` in `__stored_args__`, which you can use for display, logging, and so forth.

In [ ]:
test_eq(t.__stored_args__, {'a':1, 'b':3, 'c':2})

Since you normally want to use the first argument (often called `self`) for storing attributes, it's optional:

In [ ]:
class T:
    def __init__(self, a,b,c:str): store_attr('a,b,c')

t = T(1,c=2,b=3)
assert t.a==1 and t.b==3 and t.c==2

In [ ]:
#hide
class _T:
    def __init__(self, a,b):
        c = 2
        store_attr('a,b,c')

t = _T(1,b=3)
assert t.a==1 and t.b==3 and t.c==2

With `cast=True` any parameter annotations will be used as preprocessing functions for the corresponding arguments:

In [ ]:
class T:
    def __init__(self, a:listify, b, c:str): store_attr('a,b,c', cast=True)

t = T(1,c=2,b=3)
assert t.a==[1] and t.b==3 and t.c=='2'

You can inherit from a class using `store_attr`, and just call it again to add in any new attributes added in the derived class:

In [ ]:
class T2(T):
    def __init__(self, d, **kwargs):
        super().__init__(**kwargs)
        store_attr('d')

t = T2(d=1,a=2,b=3,c=4)
assert t.a==2 and t.b==3 and t.c==4 and t.d==1

You can skip passing a list of attrs to store. In this case, all arguments passed to the method are stored:

In [ ]:
class T:
    def __init__(self, a,b,c): store_attr()

t = T(1,c=2,b=3)
assert t.a==1 and t.b==3 and t.c==2

In [ ]:
class T4(T):
    def __init__(self, d, **kwargs):
        super().__init__(**kwargs)
        store_attr()

t = T4(4, a=1,c=2,b=3)
assert t.a==1 and t.b==3 and t.c==2 and t.d==4

In [ ]:
#hide
# ensure that subclasses work with or without `store_attr`
class T4(T):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        store_attr()

t = T4(a=1,c=2,b=3)
assert t.a==1 and t.b==3 and t.c==2

class T4(T): pass

t = T4(a=1,c=2,b=3)
assert t.a==1 and t.b==3 and t.c==2

You can skip some attrs by passing `but`:

In [ ]:
class T:
    def __init__(self, a,b,c): store_attr(but=['a'])

t = T(1,c=2,b=3)
assert t.b==3 and t.c==2
assert not hasattr(t,'a')

You can also pass keywords to `store_attr`, which is identical to setting the attrs directly, but also stores them in `__stored_args__`.

In [ ]:
class T:
    def __init__(self): store_attr(a=1)

t = T()
assert t.a==1

You can also use store_attr inside functions.

In [ ]:
def create_T(a, b):
    t = SimpleNamespace()
    store_attr(self=t)
    return t

t = create_T(a=1, b=2)
assert t.a==1 and t.b==2

In [ ]:
#export
def attrdict(o, *ks):
    "Dict from each `k` in `ks` to `getattr(o,k)`"
    return {k:getattr(o,k) for k in ks}

In [ ]:
class T:
    def __init__(self, a,b,c): store_attr()

t = T(1,c=2,b=3)
test_eq(attrdict(t,'b','c'), {'b':3, 'c':2})

In [ ]:
#export
def properties(cls, *ps):
    "Change attrs in `cls` with names in `ps` to properties"
    for p in ps: setattr(cls,p,property(getattr(cls,p)))

In [ ]:
class T:
    def a(self): return 1
    def b(self): return 2
properties(T,'a')

test_eq(T().a,1)
test_eq(T().b(),2)

In [ ]:
#export
_camel_re1 = re.compile('(.)([A-Z][a-z]+)')
_camel_re2 = re.compile('([a-z0-9])([A-Z])')

In [ ]:
#export
def camel2snake(name):
    "Convert CamelCase to snake_case"
    s1   = re.sub(_camel_re1, r'\1_\2', name)
    return re.sub(_camel_re2, r'\1_\2', s1).lower()

In [ ]:
test_eq(camel2snake('ClassAreCamel'), 'class_are_camel')
test_eq(camel2snake('Already_Snake'), 'already__snake')

In [ ]:
#export
def snake2camel(s):
    "Convert snake_case to CamelCase"
    return ''.join(s.title().split('_'))

In [ ]:
test_eq(snake2camel('a_b_cc'), 'ABCc')

In [ ]:
#export
def class2attr(self, cls_name):
    "Return the snake-cased name of the class; strip ending `cls_name` if it exists."
    return camel2snake(re.sub(rf'{cls_name}$', '', self.__class__.__name__) or cls_name.lower())

In [ ]:
class Parent:
    @property
    def name(self): return class2attr(self, 'Parent')

class ChildOfParent(Parent): pass
class ParentChildOf(Parent): pass

p = Parent()
cp = ChildOfParent()
cp2 = ParentChildOf()

test_eq(p.name, 'parent')
test_eq(cp.name, 'child_of')
test_eq(cp2.name, 'parent_child_of')

In [ ]:
#export
def hasattrs(o,attrs):
    "Test whether `o` contains all `attrs`"
    return all(hasattr(o,attr) for attr in attrs)

In [ ]:
assert hasattrs(1,('imag','real'))
assert not hasattrs(1,('imag','foo'))

In [ ]:
#export
def setattrs(dest, flds, src):
    f = dict.get if isinstance(src, dict) else getattr
    flds = re.split(r",\s*", flds)
    for fld in flds: setattr(dest, fld, f(src, fld))

In [ ]:
d = dict(a=1,bb="2",ignore=3)
o = SimpleNamespace()
setattrs(o, "a,bb", d)
test_eq(o.a, 1)
test_eq(o.bb, "2")

In [ ]:
d = SimpleNamespace(a=1,bb="2",ignore=3)
o = SimpleNamespace()
setattrs(o, "a,bb", d)
test_eq(o.a, 1)
test_eq(o.bb, "2")

In [ ]:
#export
def try_attrs(obj, *attrs):
    "Return first attr that exists in `obj`"
    for att in attrs:
        try: return getattr(obj, att)
        except: pass
    raise AttributeError(attrs)

In [ ]:
test_eq(try_attrs(1, 'real'), 1)
test_eq(try_attrs(1, 'foobar', 'real'), 1)

## Extensible Types

`ShowPrint` is a base class that defines a `show` method, which is used primarily for callbacks in fastai that expect this method to be defined.

In [ ]:
#export
#hide
class ShowPrint:
    "Base class that prints for `show`"
    def show(self, *args, **kwargs): print(str(self))

`Int`, `Float`, and `Str` extend `int`, `float` and `str` respectively by adding an additional `show` method by inheriting from `ShowPrint`.

The code for `Int` is shown below:

In [ ]:
#export
#hide
class Int(int,ShowPrint):
    "An extensible `int`"
    pass

In [ ]:
#export 
#hide
class Str(str,ShowPrint):
    "An extensible `str`"
    pass
class Float(float,ShowPrint):
    "An extensible `float`"
    pass

Examples:

In [ ]:
Int(0).show()
Float(2.0).show()
Str('Hello').show()

0
2.0
Hello


## Collection functions

Functions that manipulate popular python collections.

In [ ]:
#export
def detuplify(x):
    "If `x` is a tuple with one thing, extract it"
    return None if len(x)==0 else x[0] if len(x)==1 and getattr(x, 'ndim', 1)==1 else x

In [ ]:
test_eq(detuplify(()),None)
test_eq(detuplify([1]),1)
test_eq(detuplify([1,2]), [1,2])
test_eq(detuplify(np.array([[1,2]])), np.array([[1,2]]))

In [ ]:
#export
def replicate(item,match):
    "Create tuple of `item` copied `len(match)` times"
    return (item,)*len(match)

In [ ]:
t = [1,1]
test_eq(replicate([1,2], t),([1,2],[1,2]))
test_eq(replicate(1, t),(1,1))

In [ ]:
# export
def setify(o): 
    "Turn any list like-object into a set."
    return o if isinstance(o,set) else set(listify(o))

In [ ]:
# test
test_eq(setify(None),set())
test_eq(setify('abc'),{'abc'})
test_eq(setify([1,2,2]),{1,2})
test_eq(setify(range(0,3)),{0,1,2})
test_eq(setify({1,2}),{1,2})

In [ ]:
#export
def merge(*ds):
    "Merge all dictionaries in `ds`"
    return {k:v for d in ds if d is not None for k,v in d.items()}

In [ ]:
test_eq(merge(), {})
test_eq(merge(dict(a=1,b=2)), dict(a=1,b=2))
test_eq(merge(dict(a=1,b=2), dict(b=3,c=4), None), dict(a=1, b=3, c=4))

In [ ]:
#export
def range_of(x):
    "All indices of collection `x` (i.e. `list(range(len(x)))`)"
    return list(range(len(x)))

In [ ]:
test_eq(range_of([1,1,1,1]), [0,1,2,3])

In [ ]:
#export
def groupby(x, key, val=noop):
    "Like `itertools.groupby` but doesn't need to be sorted, and isn't lazy, plus some extensions"
    if   isinstance(key,int): key = itemgetter(key)
    elif isinstance(key,str): key = attrgetter(key)
    if   isinstance(val,int): val = itemgetter(val)
    elif isinstance(val,str): val = attrgetter(val)
    res = {}
    for o in x: res.setdefault(key(o), []).append(val(o))
    return res

In [ ]:
test_eq(groupby('aa ab bb'.split(), itemgetter(0)), {'a':['aa','ab'], 'b':['bb']})

Here's an example of how to *invert* a grouping, using an `int` as `key` (which uses `itemgetter`; passing a `str` will use `attrgetter`), and using a `val` function:

In [ ]:
d = {0: [1, 3, 7], 2: [3], 3: [5], 4: [8], 5: [4], 7: [5]}
groupby(((o,k) for k,v in d.items() for o in v), 0, 1)

{1: [0], 3: [0, 2], 7: [0], 5: [3, 7], 8: [4], 4: [5]}

In [ ]:
#export
def last_index(x, o):
    "Finds the last index of occurence of `x` in `o` (returns -1 if no occurence)"
    try: return next(i for i in reversed(range(len(o))) if o[i] == x)
    except StopIteration: return -1

In [ ]:
test_eq(last_index(9, [1, 2, 9, 3, 4, 9, 10]), 5)
test_eq(last_index(6, [1, 2, 9, 3, 4, 9, 10]), -1)

In [ ]:
#export
def filter_dict(d, func):
    "Filter a `dict` using `func`, applied to keys and values"
    return {k:v for k,v in d.items() if func(k,v)}

In [ ]:
letters = {o:chr(o) for o in range(65,73)}
letters

{65: 'A', 66: 'B', 67: 'C', 68: 'D', 69: 'E', 70: 'F', 71: 'G', 72: 'H'}

In [ ]:
filter_dict(letters, lambda k,v: k<67 or v in 'FG')

{65: 'A', 66: 'B', 70: 'F', 71: 'G'}

In [ ]:
#export
def filter_keys(d, func):
    "Filter a `dict` using `func`, applied to keys"
    return {k:v for k,v in d.items() if func(k)}

In [ ]:
filter_keys(letters, lt(67))

{65: 'A', 66: 'B'}

In [ ]:
#export
def filter_values(d, func):
    "Filter a `dict` using `func`, applied to values"
    return {k:v for k,v in d.items() if func(v)}

In [ ]:
filter_values(letters, in_('FG'))

{70: 'F', 71: 'G'}

In [ ]:
#export
def cycle(o):
    "Like `itertools.cycle` except creates list of `None`s if `o` is empty"
    o = listify(o)
    return itertools.cycle(o) if o is not None and len(o) > 0 else itertools.cycle([None])

In [ ]:
test_eq(itertools.islice(cycle([1,2,3]),5), [1,2,3,1,2])
test_eq(itertools.islice(cycle([]),3), [None]*3)
test_eq(itertools.islice(cycle(None),3), [None]*3)
test_eq(itertools.islice(cycle(1),3), [1,1,1])

In [ ]:
#export
def zip_cycle(x, *args):
    "Like `itertools.zip_longest` but `cycle`s through elements of all but first argument"
    return zip(x, *map(cycle,args))

In [ ]:
test_eq(zip_cycle([1,2,3,4],list('abc')), [(1, 'a'), (2, 'b'), (3, 'c'), (4, 'a')])

In [ ]:
#export
def sorted_ex(iterable, key=None, reverse=False):
    "Like `sorted`, but if key is str use `attrgetter`; if int use `itemgetter`"
    if isinstance(key,str):   k=lambda o:getattr(o,key,0)
    elif isinstance(key,int): k=itemgetter(key)
    else: k=key
    return sorted(iterable, key=k, reverse=reverse)

In [ ]:
#export
def negate_func(f):
    "Create new function that negates result of `f`"
    def _f(*args, **kwargs): return not f(*args, **kwargs)
    return _f

In [ ]:
def f(a): return a>0
test_eq(f(1),True)
test_eq(negate_func(f)(1),False)
test_eq(negate_func(f)(a=-1),True)

In [ ]:
#export
def argwhere(iterable, f, negate=False, **kwargs):
    "Like `filter_ex`, but return indices for matching items"
    if kwargs: f = partial(f,**kwargs)
    if negate: f = negate_func(f)
    return [i for i,o in enumerate(iterable) if f(o)]

In [ ]:
#export
def filter_ex(iterable, f=noop, negate=False, gen=False, **kwargs):
    "Like `filter`, but passing `kwargs` to `f`, defaulting `f` to `noop`, and adding `negate` and `gen`"
    if f is None: f = lambda _: True
    if kwargs: f = partial(f,**kwargs)
    if negate: f = negate_func(f)
    res = filter(f, iterable)
    if gen: return res
    return list(res)

In [ ]:
#export
def range_of(a, b=None, step=None):
    "All indices of collection `a`, if `a` is a collection, otherwise `range`"
    if is_coll(a): a = len(a)
    return list(range(a,b,step) if step is not None else range(a,b) if b is not None else range(a))

In [ ]:
test_eq(range_of([1,1,1,1]), [0,1,2,3])
test_eq(range_of(4), [0,1,2,3])

In [ ]:
#export
def renumerate(iterable, start=0):
    "Same as `enumerate`, but returns index as 2nd element instead of 1st"
    return ((o,i) for i,o in enumerate(iterable, start=start))

In [ ]:
test_eq(renumerate('abc'), (('a',0),('b',1),('c',2)))

In [ ]:
#export
def first(x, f=None, negate=False, **kwargs):
    "First element of `x`, optionally filtered by `f`, or None if missing"
    x = iter(x)
    if f: x = filter_ex(x, f=f, negate=negate, gen=True, **kwargs)
    return next(x, None)

In [ ]:
test_eq(first(['a', 'b', 'c', 'd', 'e']), 'a')
test_eq(first([False]), False)
test_eq(first([False], noop), None)

In [ ]:
#export
def nested_attr(o, attr, default=None):
    "Same as `getattr`, but if `attr` includes a `.`, then looks inside nested objects"
    try:
        for a in attr.split("."): o = getattr(o, a)
    except AttributeError: return default
    return o

In [ ]:
a = SimpleNamespace(b=(SimpleNamespace(c=1)))
test_eq(nested_attr(a, 'b.c'), getattr(getattr(a, 'b'), 'c'))
test_eq(nested_attr(a, 'b.d'), None)

In [ ]:
#export
def nested_idx(coll, *idxs):
    "Index into nested collections, dicts, etc, with `idxs`"
    if not coll or not idxs: return coll
    if isinstance(coll,str) or not isinstance(coll, typing.Collection): return None
    res = coll.get(idxs[0], None) if hasattr(coll, 'get') else coll[idxs[0]] if idxs[0]<len(coll) else None
    return nested_idx(res, *idxs[1:])

In [ ]:
a = {'b':[1,{'c':2}]}
test_eq(nested_idx(a), a)
test_eq(nested_idx(a, 'b'), [1,{'c':2}])
test_eq(nested_idx(a, 'b', 1), {'c':2})
test_eq(nested_idx(a, 'b', 1, 'c'), 2)

## fastuple

A tuple with extended functionality.

In [ ]:
#export
num_methods = """
    __add__ __sub__ __mul__ __matmul__ __truediv__ __floordiv__ __mod__ __divmod__ __pow__
    __lshift__ __rshift__ __and__ __xor__ __or__ __neg__ __pos__ __abs__
""".split()
rnum_methods = """
    __radd__ __rsub__ __rmul__ __rmatmul__ __rtruediv__ __rfloordiv__ __rmod__ __rdivmod__
    __rpow__ __rlshift__ __rrshift__ __rand__ __rxor__ __ror__
""".split()
inum_methods = """
    __iadd__ __isub__ __imul__ __imatmul__ __itruediv__
    __ifloordiv__ __imod__ __ipow__ __ilshift__ __irshift__ __iand__ __ixor__ __ior__
""".split()

In [ ]:
#export
class fastuple(tuple):
    "A `tuple` with elementwise ops and more friendly __init__ behavior"
    def __new__(cls, x=None, *rest):
        if x is None: x = ()
        if not isinstance(x,tuple):
            if len(rest): x = (x,)
            else:
                try: x = tuple(iter(x))
                except TypeError: x = (x,)
        return super().__new__(cls, x+rest if rest else x)

    def _op(self,op,*args):
        if not isinstance(self,fastuple): self = fastuple(self)
        return type(self)(map(op,self,*map(cycle, args)))

    def mul(self,*args):
        "`*` is already defined in `tuple` for replicating, so use `mul` instead"
        return fastuple._op(self, operator.mul,*args)

    def add(self,*args):
        "`+` is already defined in `tuple` for concat, so use `add` instead"
        return fastuple._op(self, operator.add,*args)

def _get_op(op):
    if isinstance(op,str): op = getattr(operator,op)
    def _f(self,*args): return self._op(op,*args)
    return _f

for n in num_methods:
    if not hasattr(fastuple, n) and hasattr(operator,n): setattr(fastuple,n,_get_op(n))

for n in 'eq ne lt le gt ge'.split(): setattr(fastuple,n,_get_op(n))
setattr(fastuple,'__invert__',_get_op('__not__'))
setattr(fastuple,'max',_get_op(max))
setattr(fastuple,'min',_get_op(min))

In [ ]:
show_doc(fastuple, title_level=4)

<h4 id="fastuple" class="doc_header"><code>class</code> <code>fastuple</code><a href="" class="source_link" style="float:right">[source]</a></h4>

> <code>fastuple</code>(**`x`**=*`None`*, **\*`rest`**) :: `tuple`

A `tuple` with elementwise ops and more friendly __init__ behavior

#### Friendly init behavior

Common failure modes when trying to initialize a tuple in python:

```py
tuple(3)
> TypeError: 'int' object is not iterable
```

or 

```py
tuple(3, 4)
> TypeError: tuple expected at most 1 arguments, got 2
```

However, `fastuple` allows you to define tuples like this and in the usual way:

In [ ]:
test_eq(fastuple(3), (3,))
test_eq(fastuple(3,4), (3, 4))
test_eq(fastuple((3,4)), (3, 4))

#### Elementwise operations

In [ ]:
show_doc(fastuple.add, title_level=5)

<h5 id="fastuple.add" class="doc_header"><code>fastuple.add</code><a href="__main__.py#L21" class="source_link" style="float:right">[source]</a></h5>

> <code>fastuple.add</code>(**\*`args`**)

`+` is already defined in `tuple` for concat, so use `add` instead

In [ ]:
test_eq(fastuple.add((1,1),(2,2)), (3,3))
test_eq_type(fastuple(1,1).add(2), fastuple(3,3))
test_eq(fastuple('1','2').add('2'), fastuple('12','22'))

In [ ]:
show_doc(fastuple.mul, title_level=5)

<h5 id="fastuple.mul" class="doc_header"><code>fastuple.mul</code><a href="__main__.py#L17" class="source_link" style="float:right">[source]</a></h5>

> <code>fastuple.mul</code>(**\*`args`**)

`*` is already defined in `tuple` for replicating, so use `mul` instead

In [ ]:
test_eq_type(fastuple(1,1).mul(2), fastuple(2,2))

#### Other Elementwise Operations

Additionally, the following elementwise operations are available:
- `le`: less than
- `eq`: equal
- `gt`: greater than
- `min`: minimum of

In [ ]:
test_eq(fastuple(3,1).le(1), (False, True))
test_eq(fastuple(3,1).eq(1), (False, True))
test_eq(fastuple(3,1).gt(1), (True, False))
test_eq(fastuple(3,1).min(2), (2,1))

You can also do other elemntwise operations like negate a `fastuple`, or subtract two `fastuple`s:

In [ ]:
test_eq(-fastuple(1,2), (-1,-2))
test_eq(~fastuple(1,0,1), (False,True,False))

test_eq(fastuple(1,1)-fastuple(2,2), (-1,-1))

In [ ]:
test_eq(type(fastuple(1)), fastuple)
test_eq_type(fastuple(1,2), fastuple(1,2))
test_ne(fastuple(1,2), fastuple(1,3))
test_eq(fastuple(), ())

## Functions on Functions

Utilities for functional programming or for defining, modifying, or debugging functions. 

In [ ]:
# export
class _Arg:
    def __init__(self,i): self.i = i
arg0 = _Arg(0)
arg1 = _Arg(1)
arg2 = _Arg(2)
arg3 = _Arg(3)
arg4 = _Arg(4)

In [ ]:
#export
class bind:
    "Same as `partial`, except you can use `arg0` `arg1` etc param placeholders"
    def __init__(self, fn, *pargs, **pkwargs):
        self.fn,self.pargs,self.pkwargs = fn,pargs,pkwargs
        self.maxi = max((x.i for x in pargs if isinstance(x, _Arg)), default=-1)

    def __call__(self, *args, **kwargs):
        args = list(args)
        kwargs = {**self.pkwargs,**kwargs}
        for k,v in kwargs.items():
            if isinstance(v,_Arg): kwargs[k] = args.pop(v.i)
        fargs = [args[x.i] if isinstance(x, _Arg) else x for x in self.pargs] + args[self.maxi+1:]
        return self.fn(*fargs, **kwargs)

In [ ]:
show_doc(bind, title_level=3)

<h3 id="bind" class="doc_header"><code>class</code> <code>bind</code><a href="" class="source_link" style="float:right">[source]</a></h3>

> <code>bind</code>(**`fn`**, **\*`pargs`**, **\*\*`pkwargs`**)

Same as `partial`, except you can use [`arg0`](/basics.html#arg0) [`arg1`](/basics.html#arg1) etc param placeholders

`bind` is the same as `partial`, but also allows you to reorder positional arguments using variable name(s) `arg{i}` where i refers to the zero-indexed positional argument. `bind` as implemented currently only supports reordering of up to the first 5 positional arguments.

Consider the function `myfunc` below, which has 3 positional arguments.  These arguments can be referenced as `arg0`, `arg1`, and `arg1`, respectively.  

In [ ]:
def myfn(a,b,c,d=1,e=2): return(a,b,c,d,e)

In the below example we bind the positional arguments of `myfn` as follows:

- The second input `14`, referenced by `arg1`, is substituted for the first positional argument.
- We supply a default value of `17` for the second positional argument.
- The first input `19`, referenced by `arg0`, is subsituted for the third positional argument.  

In [ ]:
test_eq(bind(myfn, arg1, 17, arg0, e=3)(19,14), (14,17,19,1,3))

In this next example:

- We set the default value to `17` for the first positional argument.
- The first input `19` refrenced by `arg0`, becomes the second positional argument.
- The second input `14` becomes the third positional argument.
- We override the default the value for named argument `e` to `3`.

In [ ]:
test_eq(bind(myfn, 17, arg0, e=3)(19,14), (17,19,14,1,3))

This is an example of using `bind` like `partial` and do not reorder any arguments:

In [ ]:
test_eq(bind(myfn)(17,19,14), (17,19,14,1,2))

`bind` can also be used to change default values.  In the below example, we use the first input `3` to override the default value of the named argument `e`, and supply default values for the first three positional arguments:

In [ ]:
test_eq(bind(myfn, 17,19,14,e=arg0)(3), (17,19,14,1,3))

In [ ]:
#export
def map_ex(iterable, f, *args, gen=False, **kwargs):
    "Like `map`, but use `bind`, and supports `str` and indexing"
    g = (bind(f,*args,**kwargs) if callable(f)
         else f.format if isinstance(f,str)
         else f.__getitem__)
    res = map(g, iterable)
    if gen: return res
    return list(res)

In [ ]:
t = [0,1,2,3]
test_eq(map_ex(t,operator.neg), [0,-1,-2,-3])

If `f` is a string then it is treated as a format string to create the mapping:

In [ ]:
test_eq(map_ex(t, '#{}#'), ['#0#','#1#','#2#','#3#'])

If `f` is a dictionary (or anything supporting `__getitem__`) then it is indexed to create the mapping:

In [ ]:
test_eq(map_ex(t, list('abcd')), list('abcd'))

You can also pass the same `arg` params that `bind` accepts:

In [ ]:
def f(a=None,b=None): return b
test_eq(map_ex(t, f, b=arg0), range(4))

In [ ]:
# export
def compose(*funcs, order=None):
    "Create a function that composes all functions in `funcs`, passing along remaining `*args` and `**kwargs` to all"
    funcs = listify(funcs)
    if len(funcs)==0: return noop
    if len(funcs)==1: return funcs[0]
    if order is not None: funcs = sorted_ex(funcs, key=order)
    def _inner(x, *args, **kwargs):
        for f in funcs: x = f(x, *args, **kwargs)
        return x
    return _inner

In [ ]:
f1 = lambda o,p=0: (o*2)+p
f2 = lambda o,p=1: (o+1)/p
test_eq(f2(f1(3)), compose(f1,f2)(3))
test_eq(f2(f1(3,p=3),p=3), compose(f1,f2)(3,p=3))
test_eq(f2(f1(3,  3),  3), compose(f1,f2)(3,  3))

f1.order = 1
test_eq(f1(f2(3)), compose(f1,f2, order="order")(3))

In [ ]:
#export
def maps(*args, retain=noop):
    "Like `map`, except funcs are composed first"
    f = compose(*args[:-1])
    def _f(b): return retain(f(b), b)
    return map(_f, args[-1])

In [ ]:
test_eq(maps([1]), [1])
test_eq(maps(operator.neg, [1,2]), [-1,-2])
test_eq(maps(operator.neg, operator.neg, [1,2]), [1,2])

In [ ]:
#export
def partialler(f, *args, order=None, **kwargs):
    "Like `functools.partial` but also copies over docstring"
    fnew = partial(f,*args,**kwargs)
    fnew.__doc__ = f.__doc__
    if order is not None: fnew.order=order
    elif hasattr(f,'order'): fnew.order=f.order
    return fnew

In [ ]:
def _f(x,a=1):
    "test func"
    return x-a
_f.order=1

f = partialler(_f, 2)
test_eq(f.order, 1)
test_eq(f(3), -1)
f = partialler(_f, a=2, order=3)
test_eq(f.__doc__, "test func")
test_eq(f.order, 3)
test_eq(f(3), _f(3,2))

In [ ]:
class partial0:
    "Like `partialler`, but args passed to callable are inserted at started, instead of at end"
    def __init__(self, f, *args, order=None, **kwargs):
        self.f,self.args,self.kwargs = f,args,kwargs
        self.order = ifnone(order, getattr(f,'order',None))
        self.__doc__ = f.__doc__

    def __call__(self, *args, **kwargs): return self.f(*args, *self.args, **kwargs, **self.kwargs)

In [ ]:
f = partial0(_f, 2)
test_eq(f.order, 1)
test_eq(f(3), 1) # NB: different to `partialler` example

In [ ]:
#export
def instantiate(t):
    "Instantiate `t` if it's a type, otherwise do nothing"
    return t() if isinstance(t, type) else t

In [ ]:
test_eq_type(instantiate(int), 0)
test_eq_type(instantiate(1), 1)

In [ ]:
#export
def _using_attr(f, attr, x): return f(getattr(x,attr))

In [ ]:
#export
def using_attr(f, attr):
    "Change function `f` to operate on `attr`"
    return partial(_using_attr, f, attr)

In [ ]:
t = Path('/a/b.txt')
f = using_attr(str.upper, 'name')
test_eq(f(t), 'B.TXT')

### Self (with an _uppercase_ S)

A Concise Way To Create Lambdas

In [ ]:
#export
class _Self:
    "An alternative to `lambda` for calling methods on passed object."
    def __init__(self): self.nms,self.args,self.kwargs,self.ready = [],[],[],True
    def __repr__(self): return f'self: {self.nms}({self.args}, {self.kwargs})'

    def __call__(self, *args, **kwargs):
        if self.ready:
            x = args[0]
            for n,a,k in zip(self.nms,self.args,self.kwargs):
                x = getattr(x,n)
                if callable(x) and a is not None: x = x(*a, **k)
            return x
        else:
            self.args.append(args)
            self.kwargs.append(kwargs)
            self.ready = True
            return self

    def __getattr__(self,k):
        if not self.ready:
            self.args.append(None)
            self.kwargs.append(None)
        self.nms.append(k)
        self.ready = False
        return self
    
    def _call(self, *args, **kwargs):
        self.args,self.kwargs,self.nms = [args],[kwargs],['__call__']
        self.ready = True
        return self

In [ ]:
#export
class _SelfCls:
    def __getattr__(self,k): return getattr(_Self(),k)
    def __getitem__(self,i): return self.__getattr__('__getitem__')(i)
    def __call__(self,*args,**kwargs): return self.__getattr__('_call')(*args,**kwargs)

Self = _SelfCls()

In [ ]:
#export
_all_ = ['Self']

This is a concise way to create lambdas that are calling methods on an object (note the capitalization!)

`Self.sum()`, for instance, is a shortcut for `lambda o: o.sum()`.

In [ ]:
f = Self.sum()
x = np.array([3.,1])
test_eq(f(x), 4.)

# This is equivalent to above
f = lambda o: o.sum()
x = np.array([3.,1])
test_eq(f(x), 4.)

f = Self.argmin()
arr = np.array([1,2,3,4,5])
test_eq(f(arr), arr.argmin())

f = Self.sum().is_integer()
x = np.array([3.,1])
test_eq(f(x), True)

f = Self.sum().real.is_integer()
x = np.array([3.,1])
test_eq(f(x), True)

f = Self.imag()
test_eq(f(3), 0)

f = Self[1]
test_eq(f(x), 1)

`Self` is also callable, which creates a function which calls any function passed to it, using the arguments passed to `Self`:

In [ ]:
def f(a, b=3): return a+b+2
def g(a, b=3): return a*b
fg = Self(1,b=2)
list(map(fg, [f,g]))

[5, 2]

## Other Helpers

In [ ]:
#export
class PrettyString(str):
    "Little hack to get strings to show properly in Jupyter."
    def __repr__(self): return self

In [ ]:
show_doc(PrettyString, title_level=4)

<h4 id="PrettyString" class="doc_header"><code>class</code> <code>PrettyString</code><a href="" class="source_link" style="float:right">[source]</a></h4>

> <code>PrettyString</code>() :: `str`

Little hack to get strings to show properly in Jupyter.

Allow strings with special characters to render properly in Jupyter.  Without calling `print()` strings with special characters are displayed like so:

In [ ]:
with_special_chars='a string\nwith\nnew\nlines and\ttabs'
with_special_chars

'a string\nwith\nnew\nlines and\ttabs'

We can correct this with `PrettyString`:

In [ ]:
PrettyString(with_special_chars)

a string
with
new
lines and	tabs

In [ ]:
#export
def even_mults(start, stop, n):
    "Build log-stepped array from `start` to `stop` in `n` steps."
    if n==1: return stop
    mult = stop/start
    step = mult**(1/(n-1))
    return [start*(step**i) for i in range(n)]

In [ ]:
test_eq(even_mults(2,8,3), [2,4,8])
test_eq(even_mults(2,32,5), [2,4,8,16,32])
test_eq(even_mults(2,8,1), 8)

In [ ]:
#export
def num_cpus():
    "Get number of cpus"
    try:                   return len(os.sched_getaffinity(0))
    except AttributeError: return os.cpu_count()

defaults.cpus = num_cpus()

In [ ]:
num_cpus()

64

In [ ]:
#export
def add_props(f, g=None, n=2):
    "Create properties passing each of `range(n)` to f"
    if g is None: return (property(partial(f,i)) for i in range(n))
    return (property(partial(f,i), partial(g,i)) for i in range(n))

In [ ]:
class _T(): a,b = add_props(lambda i,x:i*2)

t = _T()
test_eq(t.a,0)
test_eq(t.b,2)

In [ ]:
class _T(): 
    def __init__(self, v): self.v=v
    def _set(i, self, v): self.v[i] = v
    a,b = add_props(lambda i,x: x.v[i], _set)

t = _T([0,2])
test_eq(t.a,0)
test_eq(t.b,2)
t.a = t.a+1
t.b = 3
test_eq(t.a,1)
test_eq(t.b,3)

In [ ]:
#export
def _typeerr(arg, val, typ): return TypeError(f"{arg}=={val} not {typ}")

In [ ]:
#export
def typed(f):
    "Decorator to check param and return types at runtime"
    names = f.__code__.co_varnames
    anno = f.__annotations__
    ret = anno.pop('return',None)
    def _f(*args,**kwargs):
        kw = {**kwargs}
        if len(anno) > 0:
            for i,arg in enumerate(args): kw[names[i]] = arg
            for k,v in kw.items():
                if not isinstance(v,anno[k]): raise _typeerr(k, v, anno[k])
        res = f(*args,**kwargs)
        if ret is not None and not isinstance(res,ret): raise _typeerr("return", res, ret)
        return res
    return functools.update_wrapper(_f, f)

`typed` validates argument types at **runtime**.  This is in contrast to [MyPy](http://mypy-lang.org/) which only offers static type checking.

For example, a `TypeError` will be raised if we try to pass an integer into the first argument of the below function: 

In [ ]:
@typed
def discount(price:int, pct:float): 
    return (1-pct) * price

with ExceptionExpected(TypeError): discount(100.0, .1)

We can also optionally allow multiple types by enumarating the types in a tuple as illustrated below:

In [ ]:
def discount(price:(int,float), pct:float): 
    return (1-pct) * price

assert 90.0 == discount(100.0, .1)

In [ ]:
@typed
def foo(a:int, b:str='a'): return a
test_eq(foo(1, '2'), 1)

with ExceptionExpected(TypeError): foo(1,2)

@typed
def foo()->str: return 1
with ExceptionExpected(TypeError): foo()

@typed
def foo()->str: return '1'
assert foo()

## Notebook functions

In [ ]:
show_doc(ipython_shell)

<h4 id="ipython_shell" class="doc_header"><code>ipython_shell</code><a href="https://github.com/fastai/fastcore/tree/master/fastcore/imports.py#L66" class="source_link" style="float:right">[source]</a></h4>

> <code>ipython_shell</code>()

Same as `get_ipython` but returns `False` if not in IPython

In [ ]:
show_doc(in_ipython)

<h4 id="in_ipython" class="doc_header"><code>in_ipython</code><a href="https://github.com/fastai/fastcore/tree/master/fastcore/imports.py#L71" class="source_link" style="float:right">[source]</a></h4>

> <code>in_ipython</code>()

Check if code is running in some kind of IPython environment

In [ ]:
show_doc(in_colab)

<h4 id="in_colab" class="doc_header"><code>in_colab</code><a href="https://github.com/fastai/fastcore/tree/master/fastcore/imports.py#L75" class="source_link" style="float:right">[source]</a></h4>

> <code>in_colab</code>()

Check if the code is running in Google Colaboratory

In [ ]:
show_doc(in_jupyter)

<h4 id="in_jupyter" class="doc_header"><code>in_jupyter</code><a href="https://github.com/fastai/fastcore/tree/master/fastcore/imports.py#L79" class="source_link" style="float:right">[source]</a></h4>

> <code>in_jupyter</code>()

Check if the code is running in a jupyter notebook

In [ ]:
show_doc(in_notebook)

<h4 id="in_notebook" class="doc_header"><code>in_notebook</code><a href="https://github.com/fastai/fastcore/tree/master/fastcore/imports.py#L84" class="source_link" style="float:right">[source]</a></h4>

> <code>in_notebook</code>()

Check if the code is running in a jupyter notebook

These variables are availabe as booleans in `fastcore.basics` as `IN_IPYTHON`, `IN_JUPYTER`, `IN_COLAB` and `IN_NOTEBOOK`.

In [ ]:
IN_IPYTHON, IN_JUPYTER, IN_COLAB, IN_NOTEBOOK

(True, True, False, True)

# Export -

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_test.ipynb.
Converted 01_basics.ipynb.
Converted 02_foundation.ipynb.
Converted 03_xtras.ipynb.
Converted 04_dispatch.ipynb.
Converted 05_transform.ipynb.
Converted 06_logargs.ipynb.
Converted 07_meta.ipynb.
Converted 08_script.ipynb.
Converted index.ipynb.
